# Latam Airlines' Data Scientist Challenge Solution

- Author: Javier Welch (welch.javier[at]gmail.com)
- Date: January 13th, 2023

### Contents

0. Imports and Preprocessing
1. Question 1: How is the data distributed? Did you find any noteworthy insight to share? What can you conclude about this?
2. Question 2: Generate additional columns and export them to a CSV file.
3. Question 3: What is the behavior of the delay rate across destination, airline, month of the year, day of the week, season, type of flight?What variables would you expect to have the most influence in predicting delays?
4. Question 4: Train one or several models (using the algorithm(s) of your choice) to estimate the likelihood of a flight delay.
5. Question 5: Evaluate model performance in the predictive task across each model that you trained. Define and justify what metrics you used to assess model performance. Pick the best trained model and evaluate the following: What variables were the most influential in the prediction task? How could you improve the Performance?

## 0. Imports and Preprocessing

In [10]:
# Library imports
import pandas as pd

In [11]:
# Data import
data = pd.read_csv('../data/dataset_SCL.csv')

/var/folders/d6/r774t3wj5s77bz4nbvb4xwr40000gn/T/ipykernel_82861/2604435694.py:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/dataset_SCL.csv')


In [12]:
#Check data structure
data.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [15]:
#Check data types and non-nulls
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  int64 
 11  MES        68206 non-null  int64 
 12  AÑO        68206 non-null  int64 
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


In [19]:
#Check for extra missind values
data.isna().sum()

Fecha-I      0
Vlo-I        0
Ori-I        0
Des-I        0
Emp-I        0
Fecha-O      0
Vlo-O        1
Ori-O        0
Des-O        0
Emp-O        0
DIA          0
MES          0
AÑO          0
DIANOM       0
TIPOVUELO    0
OPERA        0
SIGLAORI     0
SIGLADES     0
dtype: int64

There are 2 date columns that should be parsed ("Fecha-I" & "Fecha-O") and a missing value in the "Vlo-O" column.

In [18]:
#Parse date columns
date_columns = ["Fecha-I", "Fecha-O"]
for col in date_columns:
    data[col] = pd.to_datetime(data[col])

In [20]:
#Identify missing value
#Flight Operation Number can be identified by 
row_missing_value = data[data['Vlo-O'].isna()]
row_missing_value

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
6068,2017-01-19 11:00:00,200,SCEL,SPJC,LAW,2017-01-19 11:03:00,NaN,SCEL,SPJC,56R,19,1,2017,Jueves,I,Latin American Wings,Santiago,Lima


In [31]:
#Missing Flight Operation Number corresponds to a LAW flight from SCEL to SPJC scheduled for Thursdays at 11:00, let's check if there's a matching record
matching_records = data[(data['Fecha-I'].dt.hour == row_missing_value['Fecha-I'].dt.hour.iloc[0]) 
    & (data['Emp-I'] == row_missing_value['Emp-I'].iloc[0]) 
    & (data['Ori-O'] == row_missing_value['Ori-O'].iloc[0]) 
    & (data['Des-O'] == row_missing_value['Des-O'].iloc[0]) 
    & (data['DIANOM'] == row_missing_value['DIANOM'].iloc[0]) 
    & ~(data['Vlo-O'].isna())]

matching_records['Vlo-O'].value_counts()

200    26
Name: Vlo-O, dtype: int64

In [32]:
#Flight Operation Number was supposed to be 200, let's patch it
data.loc[row_missing_value.index, 'Vlo-O'] = matching_records['Vlo-O'].value_counts().index[0]
data.loc[row_missing_value.index, 'Vlo-O']

6068    200
Name: Vlo-O, dtype: object